In [ ]:
# Import packages
import numpy as np
import pandas as pd

In [ ]:
filepath = "Data/{filename}.csv" # TO UPDATE after feature engineering is complete

# Column names from feature engineered dataset
cat_col_names = [] # Categorical col 
num_col_names = [] # Numerical col